Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Tianxiao Hu"
COLLABORATORS = ""

---

# Lab 09: Logistic regression 

**Collaboration Policy**

Data science is a collaborative activity. While you may talk with others about
the homework, we ask that you **write your solutions individually**. If you do
discuss the assignments with others please **include their names** at the top
of your solution.

## Due Date

This assignment is due at 11:59pm Monday, October 29nd.

In this lab, you will practice logistic regression.

# Collaborators  

Write names in this cell:

In [ ]:
# Run this cell to set up your notebook
import numpy as np
import pandas as pd
import sklearn
import sklearn.datasets
import sklearn.linear_model
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import cufflinks as cf


%matplotlib inline
sns.set()
sns.set_context("talk")
py.init_notebook_mode(connected=False)
cf.set_config_file(offline=False, world_readable=True, theme='ggplot')

In this lab, we will be working on the breast cancer dataset. This dataset can be easily loaded using the `sklearn.datasets.load_breast_cancer()` method.  
The data format is not a `pandas.DataFrame` so we will create a new DataFrame from it.

In [ ]:
data = sklearn.datasets.load_breast_cancer()
# data is actually a dictionnary
print(data.keys())
print(data.DESCR)

In [ ]:
df = pd.DataFrame(data.data, columns=data.feature_names)
df.head()

Let us try to fit a simple model with only one feature.

In [ ]:
# Define our features/target
X = df[["mean radius"]]
# Target data['target'] = 0 is malignant 1 is benign
Y = (data.target == 0)

In [ ]:
# Split between train and test
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test = train_test_split(X,Y, test_size=0.25, random_state=42)

print(f"Training Data Size: {len(x_train)}")
print(f"Test Data Size: {len(x_test)}")

### Question 1

Now let us first fit a logistic regression model using the training set.  
We will compute the training and testing accuracy, defined as:

$$
\large
\text{Training Accuracy} = \frac{1}{n_{train\_set}} \sum_{i \in \text{train_set}} {\mathbb{1}_{y_i == \hat{y_i}}}
$$

$$
\large
\text{Testing Accuracy} = \frac{1}{n_{test\_set}} \sum_{i \in \text{test_set}} {\mathbb{1}_{y_i == \hat{y_i}}}
$$

where $ y_i $ is the prediction of our model, $ y_i $ the ground truth and $\mathbb{1}_{y_i == \hat{y_i}}$ an indicator function ($ \mathbb{1}_{y_i == \hat{y_i}} = 1 \iff  y_i = \hat{y_i}$).

In [ ]:
lr = sklearn.linear_model.LogisticRegression(fit_intercept=True)
lr.fit(x_train, y_train)
train_accuracy = sum(y_train == lr.predict(x_train)) / len(y_train)
test_accuracy = sum(y_test == lr.predict(x_test)) / len(y_test)

# YOUR CODE HERE
# raise NotImplementedError()

print(f"Train accuracy: {train_accuracy:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")

In [ ]:
assert sum(lr.predict(x_test) == y_test) == 131

### Question 2
It seems we can a get very high test accuracy. Then how about precision and recall?  
- Precision (also called positive predictive value) is the fraction of relevant instances among the retrieved instances.  
- Recall (also known as sensitivity) is the fraction of relevant instances that have been retrieved over the total amount of relevant instances.

Precision is the ability of the classifier not to label as positive a sample that is negative while recall is the ability of the classifier to find all the positive samples.

To understand the link between recall/precision on the one hand and sensitivity/specificity on the other hand, it's useful to come back to a confusion matrix:

In [ ]:
from sklearn.metrics import confusion_matrix

cnf_matrix = confusion_matrix(y_test, lr.predict(x_test))

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
class_names = ['False', 'True']
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

Then:
$$
\text{Precision} = \frac{n_{true\_positives}}{n_{true\_positives} + n_{false\_positives}}
$$

$$
\text{Recall} = \frac{n_{true\_positives}}{n_{true\_positives} + n_{false\_negatives}}
$$

As illustrated in the figure below:
![precision_recall](precision_recall.png)

Now let's compute the precision and recall for the test set using the model we got from question 1.  
Please do not use the `sklearn.metrics` for this computation.

In [ ]:
np.bitwise_and([True, True], [True, False])

In [ ]:
y_pred = lr.predict(x_test) 
precision = np.sum(np.bitwise_and(y_pred == 1, y_test == 1)) / np.sum(y_pred == 1)
recall = np.sum(np.bitwise_and(y_pred == 1, y_test == 1)) / np.sum(y_test == 1)

# YOUR CODE HERE
# raise NotImplementedError()

print(f'precision = {precision:.4f}')
print(f'recall = {recall:.4f}')

In [ ]:
assert np.isclose(precision, 0.97727272727272729)
assert np.isclose(recall, 0.79629629629629628)

What can you infer from these results? Please consider the following plots, that display the distribution of the target variable in the training and testing sets.

In [ ]:
fig, axes = plt.subplots(1, 2)
sns.countplot(y_train, ax=axes[0]);
sns.countplot(y_test, ax=axes[1]);

axes[0].set_title('Train')
axes[1].set_title('Test')
plt.tight_layout();

In [ ]:
#Write your solution below:
answers = """
In dataset, false target variable are more than true target variable. So our model tend to predict more false target in order to get more accuracy.
"""
# YOUR CODE HERE
# raise NotImplementedError()

###  Question 3
Now let's try to analyze the cross entropy loss. Recall that loss would be:
$$\Large L(\theta) = -\frac{1}{n} \sum_{i=1}^n \left( y_i \phi(x_i)^T \theta + \log \left(\sigma\left(-\phi(x_i)^T \theta\right) \right) \right) $$

In [ ]:
theta = np.array([lr.coef_[0][0],
                  lr.intercept_[0]])
Phi = np.hstack([X,
                 np.ones([len(X), 1])])
print(theta)
print()
print(Phi)

In [ ]:
theta.shape

In [ ]:
def lr_loss(theta, Phi, Y):
    '''
    Compute the cross entropy loss using Phi, Y and theta. Hint: # The notation B @ v means: 
    compute the matrix multiplication Bv 

    Args:
        theta: The model parameters. 
        Phi: The transformed input data \phi(X)
        Y: The label 

    Return:
        The cross entropy loss.
    '''
    t = Phi @ theta
    loss = - np.sum(Y * t - np.log(1 + np.exp(t)), axis=0) / Y.shape[0]

#     YOUR CODE HERE
#     raise NotImplementedError()

    return loss

In [ ]:
assert np.isclose(lr_loss(np.array([0,0]),
                          Phi,
                          Y),
                  0.69314718055994529)
assert np.isclose(lr_loss(theta,
                          Phi,
                          Y),
                  0.34566168837055461)

In [ ]:
uvalues = np.linspace(-8,8,70)
vvalues = np.linspace(-5,5,70)
(u,v) = np.meshgrid(uvalues, vvalues)
thetas = np.vstack((u.flatten(),v.flatten()))
lr_loss_values = np.array([lr_loss(t, Phi, Y) for t in thetas.T])
lr_loss_surface = go.Surface(name="Logistic Regression Loss",
        x=u, y=v, z=np.reshape(lr_loss_values,(len(uvalues), len(vvalues))),
        contours=dict(z=dict(show=True, color="gray", project=dict(z=True)))
    )

py.iplot(go.Figure(data=[lr_loss_surface]))

What remarks can you make on this plot?
Solution: One remark can be made is that this plot shows that there are multiple points that minimize the loss. Therefore, you cannot uniquely define the theta.

## Submission

You're done!

Before submitting this assignment, ensure to:

1. Restart the Kernel (in the menubar, select Kernel->Restart & Run All)
2. Validate the notebook by clicking the "Validate" button

Finally, make sure to **submit** the assignment via the Assignments tab in Datahub